In [1]:
import nupack as nu, numpy as np, scipy.linalg as la
import local_time as lt

np.set_printoptions(linewidth=160)

In [2]:
A = np.random.random(size=(10, 10))
A = A @ A.T
X = nu.numeric.maximize_trace(A @ A, A, 4).solution

In [3]:
def variance_trace(A, B, X, inv=la.pinv):
    return ((X.T @ A @ X) * inv(X.T @ B @ X)).sum()

def variance_objective(A, B, X):
    '''Maximize this!'''
    return variance_trace(A, B, X)

def stationary_variance(A, mu, inv=la.pinv):
    return np.diagonal(A - np.outer(mu, mu) / (mu @ inv(A) @ mu)).sum()

In [4]:
def rates_from_local_times(mu, G, inv=la.pinv):
    alpha = mu @ inv(G) @ mu
    # assert np.all(inv(G) @ mu > 0), inv(G) @ mu
    pi = alpha**-1 * np.diag(mu) @ inv(G) @ mu
    R = np.diag(pi**-1) @ (np.diag(mu) @ inv(G) @ np.diag(mu) - alpha * np.outer(pi, pi))
    G = alpha**-2 * np.diag(inv(G) @ mu) @ G @ np.diag(inv(G) @ mu)   - alpha**-1 * np.outer(pi, pi.T)
    return pi, R, G

In [5]:
variance_trace(A @ A, A, X)

28.190277844603997

In [6]:
np.diagonal(A).sum()

37.317460738748075

In [7]:
variance_trace(A @ A, A, np.random.random(size=(10, 4)))

34.60977927945805

In [8]:
strands = ('GCGUCGCGUCGCUAUGC',)

In [9]:
model = nu.Model()
# model.rate_function.cast(nu.model.Kawasaki).unimolecular_scaling = 1

analysis = nu.Analysis(model=model)
analysis.boltzmann_sample(strands, 1e4) # generates set # of sample structures within the boltzmann distribution
analysis.pair_probability(strands)

results = analysis.compute() # compute partition function, MFE, PP matrix, etc. for each sequence in boltzmann sample

P = results[strands].pair_probability
free_energy = results[strands].free_energy
samples = results[strands].sampled_structures
bps = lt.generate_locs(P, 10)
tau = 1e-3
means = P[bps]

In [10]:
def sample_times(structure):
    p = nu.path.sample_path(nu.State(strands, structure.pairs, model=model), tau)
    return p.base_pair_time()[bps] - p.time * means, p.time
    
from concurrent.futures import ThreadPoolExecutor

with ThreadPoolExecutor(4) as ex:
    paths, times = zip(*ex.map(sample_times, samples))

total_time = sum(times)
paths = np.array(paths)

Gamma = paths.T @ paths / total_time

In [11]:
# _, _, Gamma = rates_from_local_times(means, Gamma)

X = nu.numeric.maximize_trace(Gamma @ Gamma, Gamma, 2, ops=nu.numeric.MaxTraceOptions(tolerance=1e-8)).solution
mu = means @ X
G = X.T @ Gamma @ X

print(X)
print('Variance', variance_objective(Gamma, X))
print('Stationary variance', stationary_variance(G, mu))
print('Alpha %e' % (mu @ la.pinv(G) @ mu))
print()

# X = np.array([(m < 0.5, m >= 0.5) for m in means], dtype=np.float64)
# mu = means @ X
# G = X.T @ Gamma @ X

# print(X)
# print('Variance', variance_objective(Gamma, X))
# print('Stationary variance', stationary_variance(G, mu))
# print('Alpha %e' % (mu @ la.pinv(G) @ mu))

RuntimeError: C++: : "Assertion failure" (MaxTrace.h, line 31)
quadratic_positive_definite(a, b, c) = false
a = nan
b = nan
c = nan


In [ ]:
vals, vecs = la.eig(la.pinv(G))
print(vals)
print(vecs)

print(mu @ la.pinv(G) @ [0.85647148, 0.51619435])

In [ ]:
ok = 0
for _ in range(100):
    h = np.random.random(size=2)
    if np.all(la.pinv(G) @ h > 0):
        ok += 1
print(ok)

In [ ]:
for m, x in zip(means, X):
    print(x, m)

In [ ]:
pi, R, G2 = rates_from_local_times(mu, G)

print('Local time answer')
print(pi)
print(R)
print(G2)

# Enumerated version

In [12]:
states, pi0, R0, _ = nu.State(strands, model=model).rate_matrix()
G0 = np.diag(pi0) @ lt.Z_from_R(pi0, R0.toarray())

bfs0 = np.array([[(w.pairs[i+1] == j+1) for i, j in zip(*bps)] for w in states], dtype=np.float64)
print(np.diagonal(bfs0.T @ G0 @ bfs0).sum())

-84121314160.54843
0.0005645775574231974


In [13]:
which = np.argsort(pi0)[-2:]
pi, R, G2 = rates_from_local_times(pi0[which], G0[which][:, which])

print('Enumerated answer for microstates')
print(pi)
print(R)
print(G2)

Enumerated answer for microstates
[0.10636573 0.89363427]
[[ 1385.88109335 -1385.88109335]
 [ -164.95591711   164.95591711]]
[[ 6.12908153e-05 -6.12908153e-05]
 [-6.12908153e-05  6.12908153e-05]]


In [14]:
which = np.argsort(pi0)[-10:]
G = G0[which][:, which]
G2 = (G0 @ np.diag(1/pi0) @ G0)[which][:, which]
pi = pi0[which]

# G = G - np.outer(pi, pi) / (pi @ la.pinv(G) @ pi)
# G2 = G2 - np.outer(pi, pi) / (pi @ la.pinv(G2) @ pi)

result = nu.numeric.maximize_trace(G2, G, 2, ops=nu.numeric.MaxTraceOptions(tolerance=1e-4, pseudoinverse=True))
X = result.solution

print(X)
print(pi @ X)

[[0.063652   0.13791902]
 [0.09872658 0.10194819]
 [0.14769936 0.1186872 ]
 [0.11877314 0.10249912]
 [0.11452551 0.09988781]
 [0.1176664  0.10166475]
 [0.01653598 0.00037006]
 [0.06286817 0.11644467]
 [0.01744684 0.00047159]
 [0.24210605 0.22010758]]
[0.20802312 0.18886961]


In [15]:
def evaluate(X):
    print('Variance objective =', variance_objective(G2, G, X))
    print('Stationary objective =', pi @ X @ la.pinv(X.T @ G @ X) @ X.T @ pi)
    pib, R, Gb = rates_from_local_times(pi @ X, X.T @ G @ X)
    print(pib)
    print(R)
    print(Gb)
    
print('Auto')
evaluate(X)

print('Manual')
Y = 0 * X
Y[:2] = np.eye(2)

evaluate(Y)

print('Eigenvectors')
vals, vecs = la.eigh(G2, G)
evaluate(vecs[:, -2:])

Auto
Variance objective = 0.0007446921156983866
Stationary objective = 16511.50779247956
[ 4.03266323 -3.03266323]
[[ 666609.32568057 -666609.32568057]
 [ 886419.20098912 -886419.20098912]]
[[ 5.56376708e-05 -5.56376708e-05]
 [-5.56376708e-05  5.56376708e-05]]
Manual
Variance objective = 0.0003990454292602941
Stationary objective = 210.30843616626737
[0.31981329 0.68018671]
[[ 348.77384641 -348.77384641]
 [-163.98808036  163.98808036]]
[[ 0.00042424 -0.00042424]
 [-0.00042424  0.00042424]]
Eigenvectors
Variance objective = 0.0007454507029326465
Stationary objective = 3868.7399257025545
[0.69408377 0.30591623]
[[ 1183.51031473 -1183.51031473]
 [-2685.22961098  2685.22961098]]
[[ 5.48838879e-05 -5.48838879e-05]
 [-5.48838879e-05  5.48838879e-05]]
